In [ ]:
!pip install enelvo

In [ ]:
from google.colab import drive
from enelvo.normaliser import Normaliser
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
drive.mount('/content/drive')

# Pré-processamento

## 0) Base de Dados

In [ ]:
norm = Normaliser(tokenizer='readable')
chats_clients = pd.read_csv('/content/drive/Shareddrives/grupo3moshi/SPRINT_3/data/data_sprint_3.csv')
chats_clients

In [ ]:
chats_clients.fillna('erro ao processar a pergunta', inplace=True)

In [ ]:
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def shuffle_words(sentence):
    """
    Embaralha as palavras em uma frase
    """
    words = sentence.split()
    random.shuffle(words)
    return ' '.join(words)

def remove_stopwords(sentence):
    """
    Remove as stopwords em uma frase
    """
    stop_words = set(stopwords.words('portuguese'))
    word_tokens = word_tokenize(sentence)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    return ' '.join(filtered_sentence)

# Aplicando o embaralhamento de palavras
chats_clients['Pergunta_Shuffled'] = chats_clients['Pergunta'].apply(lambda x: shuffle_words(x))
chats_clients['Resposta_Shuffled'] = chats_clients['Resposta'].apply(lambda x: shuffle_words(x))


# Mostrar as primeiras linhas após as técnicas de data augmentation
chats_clients[['Pergunta', 'Pergunta_Shuffled', 'Resposta', 'Resposta_Shuffled']].head()


In [ ]:
import random
from nltk.corpus import wordnet

nltk.download('omw-1.4')
nltk.download('wordnet')

def get_synonyms(word):
    """
    Retorna os sinônimos de uma palavra
    """
    synonyms = set()
    for syn in wordnet.synsets(word, lang='por'):
        for lemma in syn.lemmas(lang='por'):
            synonyms.add(lemma.name().replace("_", " "))
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

def synonym_replacement(sentence, n=2):
    """
    Substitui aleatoriamente n palavras por seus sinônimos
    """
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if len(get_synonyms(word)) > 0]))
    random.shuffle(random_word_list)

    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    return ' '.join(new_words)

# Aplicando a substituição de sinônimos
chats_clients['Pergunta_Augmented'] = chats_clients['Pergunta'].apply(lambda x: synonym_replacement(x))
chats_clients['Resposta_Augmented'] = chats_clients['Resposta'].apply(lambda x: synonym_replacement(x))

# Mostrar as primeiras linhas após a substituição de sinônimos
chats_clients[['Pergunta', 'Pergunta_Augmented', 'Resposta', 'Resposta_Augmented']].head()


In [ ]:
def random_deletion(words, p=0.14):
    """
    Remove com a probabilidade p as palavras de uma frase
    """
    words = words.split()
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)
    return ' '.join(new_words)

# Aplicando a deleção aleatória
chats_clients['Pergunta_Deletion'] = chats_clients['Pergunta'].apply(lambda x: random_deletion(x))

# Mostrar as primeiras linhas após a deleção aleatória
chats_clients[['Pergunta_Deletion']]

In [ ]:
# Criar um novo DataFrame para armazenar os dados aumentados
df_augmented = pd.DataFrame()

# Adicionar as linhas originais
df_augmented = chats_clients.copy()

# Adicionar as linhas de perguntas e respostas embaralhadas
df_augmented_extra1 = pd.DataFrame({
    'No': chats_clients['No'],
    'Intencao': chats_clients['Intencao'],
    'Pergunta': chats_clients['Pergunta_Shuffled'],
    'Resposta': chats_clients['Resposta_Shuffled']
})

df_augmented_extra2 = pd.DataFrame({
    'No': chats_clients['No'],
    'Intencao': chats_clients['Intencao'],
    'Pergunta': chats_clients['Pergunta_Augmented'],
    'Resposta': chats_clients['Resposta_Augmented']
})

df_augmented_extra3 = pd.DataFrame({
    'No': chats_clients['No'],
    'Intencao': chats_clients['Intencao'],
    'Pergunta': chats_clients['Pergunta_Deletion'],
    'Resposta': chats_clients['Resposta']
})
# Concatenar o DataFrame original com o DataFrame de data augmentation
df_augmented = pd.concat([df_augmented, df_augmented_extra1, df_augmented_extra2, df_augmented_extra3], ignore_index=True)

# Remover as colunas auxiliares de shuffle
df_augmented = df_augmented[['No', 'Intencao', 'Pergunta', 'Resposta']]

# Exibir as primeiras linhas do DataFrame final
df_augmented.shape


(5636, 4)

In [ ]:
# Especificar o caminho onde o arquivo CSV será salvo
output_file_path = '/content/drive/Shareddrives/grupo3moshi/augmented_data.csv'

# Salvar o DataFrame aumentado como um arquivo CSV
df_augmented.to_csv(output_file_path, index=False)
